<a href="https://colab.research.google.com/github/Abhs1897/Machine-Learning/blob/main/Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import numpy as np
import torch

In [71]:
x=torch.tensor(3.)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)

In [72]:
y=w*x+b
y

tensor(17., grad_fn=<AddBackward0>)

In [73]:
#Finding gradient
y.backward()
print("dy/dw:",w.grad)
print("dy/db:",b.grad)

dy/dw: tensor(3.)
dy/db: tensor(1.)


In [74]:
#Example 1 Linear Regression

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')

In [75]:
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [76]:
print(inputs.shape)
print(targets.shape)

torch.Size([5, 3])
torch.Size([5, 2])


In [77]:
w=torch.randn(2,3,requires_grad=True) #No. of output features, No. of input features
b=torch.randn(2,requires_grad=True) # No. of output features
print(w)
print(b)

tensor([[ 0.8622, -0.0559,  1.6371],
        [-0.1209,  0.1830, -0.0977]], requires_grad=True)
tensor([ 0.0999, -0.9304], requires_grad=True)


In [78]:
def model(x):
  return x@w.t()+b

In [79]:
preds=model(inputs)
preds

tensor([[129.6962,  -1.6947],
        [178.4230,  -2.0788],
        [162.5797,   7.4096],
        [146.2202,  -9.0077],
        [168.8292,   1.4595]], grad_fn=<AddBackward0>)

In [80]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [81]:
def mse(data1,data2):
  diff=data1-data2
  return torch.sum(diff*diff)/diff.numel()

In [82]:
loss=mse(targets,preds)
print(loss)

tensor(8405.6514, grad_fn=<DivBackward0>)


In [83]:
# Compute gradients
loss.backward()

with torch.no_grad():
  w-=w.grad*1e-5
  b-=b.grad*1e-5
  w.grad.zero_()
  b.grad.zero_()

In [84]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)
#Loss would reduce

tensor(5936.3994, grad_fn=<DivBackward0>)


In [85]:
#Training for multiple epochs
for i in range(100):
  preds=model(inputs)
  loss=mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5
    w.grad.zero_()
    b.grad.zero_()

In [87]:
loss=mse(preds,targets)
print(loss)

tensor(278.4945, grad_fn=<DivBackward0>)


In [89]:
#Using Pytorch
import torch.nn as nn
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119],
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119],
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [90]:
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)

In [92]:
from torch.utils.data import TensorDataset,DataLoader
train_ds=TensorDataset(inputs,targets)
train_ds[0]

(tensor([73., 67., 43.]), tensor([56., 70.]))

In [93]:
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)
next(iter(train_dl))

[tensor([[91., 88., 64.],
         [73., 67., 43.],
         [91., 88., 64.],
         [91., 88., 64.],
         [69., 96., 70.]]),
 tensor([[ 81., 101.],
         [ 56.,  70.],
         [ 81., 101.],
         [ 81., 101.],
         [103., 119.]])]

In [94]:
model=nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2793,  0.4033,  0.1419],
        [ 0.5450,  0.0070, -0.2698]], requires_grad=True)
Parameter containing:
tensor([-0.3242,  0.5659], requires_grad=True)


In [95]:
#Optimizer
#Instead of manually manipulating the weights & biases using gradients, we can use the optimizer optim.SGD

opt=torch.optim.SGD(model.parameters(),lr=1e-5)

In [96]:
#Loss function
import torch.nn.functional as F

In [97]:
loss_fn=F.mse_loss

In [98]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(4471.3296, grad_fn=<MseLossBackward0>)


In [99]:
#Training the model
def fit(num_epochs,model,loss_fn,opt):
  for epochs in range(num_epochs):
    for xb,yb in train_dl:
      pred=model(xb)
      loss=loss_fn(pred,yb)
      loss.backward()
      opt.step()
      opt.zero_grad()
  print("Training Loss:", loss_fn(model(inputs),targets))

In [100]:
fit(100,model,loss_fn,opt)

Training Loss: tensor(40.8019, grad_fn=<MseLossBackward0>)


In [109]:
#Nueral Networks

class SimpleNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1=nn.Linear(3,3)
    self.ac1=nn.ReLU()
    self.linear2=nn.Linear(3,2)

  def forward(self,x):
    x=self.linear1(x)
    x=self.ac1(x)
    return self.linear2(x)


In [110]:
model=SimpleNet()
opt=torch.optim.SGD(model.parameters(),1e-5)
loss_fn=F.mse_loss

In [111]:
fit(100, model, loss_fn, opt)

Training Loss: tensor(23.5008, grad_fn=<MseLossBackward0>)
